In [1]:
import json
import pandas as pd
import numpy as np
import re
import matplotlib as plt

In [2]:
file_dir = "C:/Users/cthompson/Desktop/UCF/git/Movies-ETL-Practice/"

In [3]:
with open (f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [4]:
len(wiki_movies_raw)

7311

In [5]:
#first 5 records
wiki_movies_raw[3600:3605]

[{'url': 'https://en.wikipedia.org/wiki/Benji:_Off_the_Leash!',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0315273/',
  'title': 'Benji: Off the Leash!',
  'Directed by': 'Joe Camp',
  'Written by': 'Joe Camp',
  'Starring': ['Benji', 'Nick Whitaker', 'Shaggy', 'Gypsy the Cockatoo'],
  'Music by': 'Antonio di Lorenzo',
  'Productioncompany ': 'Mulberry Square Productions',
  'Distributed by': 'Mulberry Square Productions',
  'Release date': ['March 26, 2004', '(', '2004-03-26', ')'],
  'Running time': '97 min',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$3,817,362'},
 {'url': 'https://en.wikipedia.org/wiki/The_Best_Thief_in_the_World',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0389796/',
  'title': 'The Best Thief in the World',
  'Directed by': 'Jacob Kornbluth',
  'Produced by': ['Tim Perrell', 'Nicola Usborne'],
  'Written by': 'Jacob Kornbluth',
  'Starring': ['Marc Rozendaal',
   'Michael Silverman',
   'David Warsh

In [6]:
kaggle_metadata = pd.read_csv(f'{file_dir}/movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}/ratings.csv')

In [7]:
ratings.sample(n=5)

,userId,movieId,rating,timestamp
24303697,252403,7153,3.5,1128471694
19291531,200365,3535,3.0,1447601899
25317181,263594,34319,3.5,1230778888
16510232,171634,1479,4.0,1453267029
3566390,37168,6502,5.0,1342058057


In [8]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
wiki_movies_df.head()

,Actor control,Adaptation by,Alias,Alma mater,Also known as,Animation by,Arabic,Area,Area served,Artist(s),...,Website,Writer(s),Written by,Yale Romanization,Years active,Yiddish,imdb_link,title,url,year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.imdb.com/title/tt0099005/,Air America,https://en.wikipedia.org/wiki/Air_America_(film),1990.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Woody Allen,NaN,NaN,NaN,https://www.imdb.com/title/tt0099012/,Alice,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Paul Hogan,NaN,NaN,NaN,https://www.imdb.com/title/tt0099018/,Almost an Angel,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0


In [9]:
wiki_movies_df.columns.tolist()

['Actor control',
 'Adaptation by',
 'Alias',
 'Alma mater',
 'Also known as',
 'Animation by',
 'Arabic',
 'Area',
 'Area served',
 'Artist(s)',
 'Attraction type',
 'Audio format',
 'Author',
 'Based on',
 'Biographical data',
 'Bopomofo',
 'Born',
 'Box office',
 'Budget',
 'Camera setup',
 'Cantonese',
 'Characters',
 'Children',
 'Chinese',
 'Cinematography',
 'Closing date',
 'Color process',
 'Comics',
 'Composer(s)',
 'Coordinates',
 'Country',
 'Country of origin',
 'Cover artist',
 'Created by',
 'Date premiered',
 'Designer(s)',
 'Developed by',
 'Developer(s)',
 'Dewey Decimal',
 'Died',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Divisions',
 'Duration',
 'Edited by',
 'Editor(s)',
 'Ending theme',
 'Engine',
 'Engine(s)',
 'Executive producer(s)',
 'Family',
 'Fate',
 'Film(s)',
 'Followed by',
 'Format(s)',
 'Formerly',
 'Founded',
 'Founder',
 'Founders',
 'French',
 'Full name',
 'Gender',
 'Genre',
 'Genre(s)',
 'Genres',
 'Gwoyeu Romatzyh',
 'Ha

In [10]:
wiki_movies = [movie for movie in wiki_movies_raw
              if ('Director' in movie or 'Directed by' in movie)
                  and 'imdb_link' in movie]
len(wiki_movies)

7080

In [11]:
wiki_movies = [movie for movie in wiki_movies_raw
              if ('Director' in movie or 'Directed by' in movie
                 and 'imdb_link' in movie
                 and 'No. of episodes' not in movie)]
len(wiki_movies)

7076

In [12]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructiver copy
    return movie

In [13]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

7060    https://en.wikipedia.org/wiki/The_Insult_(film)
7293     https://en.wikipedia.org/wiki/Capernaum_(film)
Name: url, dtype: object

In [14]:
sorted(wiki_movies_df.columns.tolist())

['Actor control',
 'Adaptation by',
 'Alias',
 'Alma mater',
 'Also known as',
 'Animation by',
 'Arabic',
 'Area',
 'Area served',
 'Artist(s)',
 'Attraction type',
 'Audio format',
 'Author',
 'Based on',
 'Biographical data',
 'Bopomofo',
 'Born',
 'Box office',
 'Budget',
 'Camera setup',
 'Cantonese',
 'Characters',
 'Children',
 'Chinese',
 'Cinematography',
 'Closing date',
 'Color process',
 'Comics',
 'Composer(s)',
 'Coordinates',
 'Country',
 'Country of origin',
 'Cover artist',
 'Created by',
 'Date premiered',
 'Designer(s)',
 'Developed by',
 'Developer(s)',
 'Dewey Decimal',
 'Died',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Divisions',
 'Duration',
 'Edited by',
 'Editor(s)',
 'Ending theme',
 'Engine',
 'Engine(s)',
 'Executive producer(s)',
 'Family',
 'Fate',
 'Film(s)',
 'Followed by',
 'Format(s)',
 'Formerly',
 'Founded',
 'Founder',
 'Founders',
 'French',
 'Full name',
 'Gender',
 'Genre',
 'Genre(s)',
 'Genres',
 'Gwoyeu Romatzyh',
 'Ha

In [15]:
def clean_movie(movie):
    movie = dict(movie) #create a non destructive copy
    alt_titles = {}
    for key in ['Also known as', 'Arabic', 'Canonese','Chinese', 'French',
               'Hangul','Hebrew','Japanese', 'Literally', 'Mandarin', 'McCune-Reischauer', 'Original title', 'Polish',
               'Revised Romanization', 'Romanized','Russian',
               'Simplified', 'Traditional', 'Yiddish']:
        if key in movie:
            alt_titles[key]=movie[key]
            movie.pop(key)
    if len(alt_titles)>0:
        movie['alt_titles'] = alt_titles
        
    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)    
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')  
    
    return movie

In [16]:
#rerun list comprehensin to clean wiki_movies and recreate wiki_movies_df
clean_movies = [clean_movie(movie) for movie in wiki_movies]
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cantonese',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Created by',
 'Director',
 'Distributor',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Hepburn',
 'Label',
 'Language',
 'McCune–Reischauer',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Picture format',
 'Preceded by',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Recorded',
 'Release date',
 'Running time',
 'Starring',
 'Suggested by',
 'Venue',
 'Voices of',
 'Writer(s)',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

In [17]:
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset = 'imdb_id', inplace = True)
print(len(wiki_movies_df))
wiki_movies_df.head()

7076
7033


,Animation by,Audio format,Based on,Box office,Budget,Cantonese,Cinematography,Color process,Composer(s),Country,...,Suggested by,Venue,Voices of,Writer(s),alt_titles,imdb_link,title,url,year,imdb_id
0,NaN,NaN,"[Characters, by Rex Weiner]",$21.4 million,$20 million,NaN,Oliver Wood,NaN,"[Cliff Eidelman, Yello]",United States,...,NaN,NaN,NaN,"[David Arnott, James Cappe]",NaN,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,tt0098987
1,NaN,NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...",$2.7 million,$6 million,NaN,Mark Plummer,NaN,Maurice Jarre,United States,...,NaN,NaN,NaN,"[James Foley, Robert Redlin]",NaN,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,tt0098994
2,NaN,NaN,"[Air America, by, Christopher Robbins]","$57,718,089",$35 million,NaN,Roger Deakins,NaN,Charles Gross,United States,...,NaN,NaN,NaN,"[John Eskow, Richard Rush]",NaN,https://www.imdb.com/title/tt0099005/,Air America,https://en.wikipedia.org/wiki/Air_America_(film),1990,tt0099005
3,NaN,NaN,NaN,"$7,331,647",$12 million,NaN,Carlo Di Palma,NaN,NaN,United States,...,NaN,NaN,NaN,Woody Allen,NaN,https://www.imdb.com/title/tt0099012/,Alice,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,tt0099012
4,NaN,NaN,NaN,"$6,939,946 (USA)",$25 million,NaN,Russell Boyd,NaN,Maurice Jarre,US,...,NaN,NaN,NaN,Paul Hogan,NaN,https://www.imdb.com/title/tt0099018/,Almost an Angel,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,tt0099018


In [18]:
[[column, wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]

[['Animation by', 7031],
 ['Audio format', 6972],
 ['Based on', 4852],
 ['Box office', 1548],
 ['Budget', 2295],
 ['Cantonese', 7032],
 ['Cinematography', 691],
 ['Color process', 7032],
 ['Composer(s)', 518],
 ['Country', 236],
 ['Created by', 7023],
 ['Director', 0],
 ['Distributor', 357],
 ['Editor(s)', 548],
 ['Executive producer(s)', 6936],
 ['Followed by', 7024],
 ['Genre', 6923],
 ['Hepburn', 7029],
 ['Label', 7031],
 ['Language', 244],
 ['McCune–Reischauer', 7031],
 ['Narrated by', 6752],
 ['Original language(s)', 6875],
 ['Original network', 6908],
 ['Picture format', 6969],
 ['Preceded by', 7023],
 ['Producer(s)', 202],
 ['Production company(s)', 1678],
 ['Production location(s)', 6986],
 ['Recorded', 7031],
 ['Release date', 32],
 ['Running time', 139],
 ['Starring', 184],
 ['Suggested by', 7032],
 ['Venue', 7032],
 ['Voices of', 7031],
 ['Writer(s)', 199],
 ['alt_titles', 7012],
 ['imdb_link', 0],
 ['title', 1],
 ['url', 0],
 ['year', 0],
 ['imdb_id', 0]]

In [19]:
[column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df)* 0.9]

['Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Composer(s)',
 'Country',
 'Director',
 'Distributor',
 'Editor(s)',
 'Language',
 'Producer(s)',
 'Production company(s)',
 'Release date',
 'Running time',
 'Starring',
 'Writer(s)',
 'imdb_link',
 'title',
 'url',
 'year',
 'imdb_id']

In [20]:
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) *0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]


In [21]:
wiki_movies_df.dtypes

Based on                 object
Box office               object
Budget                   object
Cinematography           object
Composer(s)              object
Country                  object
Director                 object
Distributor              object
Editor(s)                object
Language                 object
Producer(s)              object
Production company(s)    object
Release date             object
Running time             object
Starring                 object
Writer(s)                object
imdb_link                object
title                    object
url                      object
year                      int64
imdb_id                  object
dtype: object

In [22]:
box_office = wiki_movies_df['Box office'].dropna()

In [23]:
len(box_office)

5485

In [24]:
def is_not_a_string(x):
    return type(x) != str
box_office[box_office.map(is_not_a_string)]

34                                       [US$, 4,212,828]
54                  [$6,698,361 (, United States, ), [2]]
74                                [$6,488,144, (US), [1]]
126                            [US$1,531,489, (domestic)]
130                                      [US$, 4,803,039]
177                                       [$3,331, (USA)]
203                      [$739,104, (North America), [2]]
210                             [$1.2 million, (US), [1]]
254                 [$14.6 million, (North America), [3]]
271                              [$38 million, (US), [2]]
278                 [$57.5 million, (North America), [1]]
340            [£739,989 (UK), [1], $4,413,473 (US), [1]]
345                            [$4,654,288 (, US, ), [1]]
378                  [$6.4 million, (North America), [1]]
414                      [$46.7 million, [4], [3], (USA)]
490                [$14.1 million, [, citation needed, ]]
514                 [$10.7 million, (North America), [2]]
534           

In [25]:
box_office[box_office.map(lambda x: type(x) !=str)]

34                                       [US$, 4,212,828]
54                  [$6,698,361 (, United States, ), [2]]
74                                [$6,488,144, (US), [1]]
126                            [US$1,531,489, (domestic)]
130                                      [US$, 4,803,039]
177                                       [$3,331, (USA)]
203                      [$739,104, (North America), [2]]
210                             [$1.2 million, (US), [1]]
254                 [$14.6 million, (North America), [3]]
271                              [$38 million, (US), [2]]
278                 [$57.5 million, (North America), [1]]
340            [£739,989 (UK), [1], $4,413,473 (US), [1]]
345                            [$4,654,288 (, US, ), [1]]
378                  [$6.4 million, (North America), [1]]
414                      [$46.7 million, [4], [3], (USA)]
490                [$14.1 million, [, citation needed, ]]
514                 [$10.7 million, (North America), [2]]
534           

In [26]:
box_office = box_office.apply(lambda x: ' ' .join(x) if type(x)== list else x)

In [27]:
box_office

0                       $21.4 million
1                        $2.7 million
2                         $57,718,089
3                          $7,331,647
4                    $6,939,946 (USA)
9                            $855,810
10                     $195.3 million
11                      $53.2 million
12                      $15.7 million
13                      $52.1 million
14                     $244.5 million
15                        $12,626,043
17                        $19,740,070
18                       $138,697,012
20                         $1,292,323
21                       $8.2 million
22                      $15.6 million
24                         $2,070,871
25      $27.6 million (United States)
27                            $10,173
29                         $5 million
33                      $35.8 million
34                      US$ 4,212,828
37                       $2.4 million
39                           $947,306
40                           $715,983
42          

In [28]:
form_one = "\$\s*\d+\.?\d*\s*[mb]illi?on"

In [29]:
box_office.str.contains(form_one, flags = re.IGNORECASE, na=False).sum()

3903

In [30]:
form_two = r"\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)"
box_office.str.contains(form_two, flags = re.IGNORECASE, na=False).sum()

1559

In [31]:
matches_form_one = box_office.str.contains(form_one, flags = re.IGNORECASE, na=False)
matches_form_two = box_office.str.contains(form_two, flags = re.IGNORECASE, na=False)

In [32]:
# this will throw an error!
box_office[(~ matches_form_one) & (~ matches_form_two)]

110              $4.35-4.37 million
600                      $5000 (US)
1070                     35,254,617
1480                     £3 million
1865                   ¥1.1 billion
2032                            N/A
2091                           $309
2665    926,423 admissions (France)
3088      $32 [2] –33.1 million [1]
3631                            TBA
3859             $38.9–40.3 million
3879       CN¥3.650 million (China)
4116                     £7,385,434
4261             $20.7–23.9 million
4306                         $20-30
4561        $45.2k (only in Turkey)
5447                          £2.56
5784                       413 733$
6013                        Unknown
6145             $17.5–18.4 million
6234             $41.8–41.9 million
6369                          $111k
6370                           $588
6593                 less than $372
6843                        8 crore
Name: Box office, dtype: object

In [33]:
box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [34]:
box_office.str.extract(f'({form_one}|{form_two})')

,0
0,$21.4 million
1,$2.7 million
2,"$57,718,089"
3,"$7,331,647"
4,"$6,939,946"
9,"$855,810"
10,$195.3 million
11,$53.2 million
12,$15.7 million
13,$52.1 million


In [35]:
def parse_dollars(s):
     # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on',s, flags=re.IGNORECASE):
        
        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]', '',s)
        # convert to float and multiply by a million
        value = float(s) * 10**6
        # return value
        return value
    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
                  
        # remove dollar sign and " billion"
        s=re.sub('\$|\s|[a-zA-Z]', '', s)
        # convert to float and multiply by a billion
        value = float(s) *10**9
        # return value
        return value
    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
        # remove dollar sign and commas
        s = re.sub('\$|,','', s)
        # convert to float
        value = float(s)
        # return value
        return value
    # otherwise, return NaN
    else:
          return np.nan

In [36]:
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [37]:
wiki_movies_df['box_office']

0       2.140000e+07
1       2.700000e+06
2       5.771809e+07
3       7.331647e+06
4       6.939946e+06
5                NaN
6                NaN
7                NaN
8                NaN
9       8.558100e+05
10      1.953000e+08
11      5.320000e+07
12      1.570000e+07
13      5.210000e+07
14      2.445000e+08
15      1.262604e+07
16               NaN
17      1.974007e+07
18      1.386970e+08
19               NaN
20      1.292323e+06
21      8.200000e+06
22      1.560000e+07
23               NaN
24      2.070871e+06
25      2.760000e+07
26               NaN
27      1.017300e+04
28               NaN
29      5.000000e+06
            ...     
7046    7.600000e+07
7047    1.206000e+08
7048    5.293000e+08
7049    2.141000e+08
7050    8.480000e+07
7051    3.230000e+08
7052    5.100000e+06
7053    9.590000e+07
7054    7.270000e+07
7055             NaN
7056    4.300000e+07
7057    7.564730e+05
7058    6.860000e+07
7059    4.660000e+07
7060    9.500000e+06
7061    3.755000e+08
7062    8.370

In [38]:
wiki_movies_df.drop('Box office', axis = 1, inplace=True)

In [39]:
budget = wiki_movies_df['Budget'].dropna()
budget = budget.map(lambda x: ''.join(x) if type(x) == list else x)

In [40]:
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [41]:
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)
budget[~matches_form_one & ~matches_form_two]

136                        Unknown
204     60 millionNorwegian Kroner
478                        Unknown
973               $34[3][4]million
1126                $120[4]million
1226                       Unknown
1278                           HBO
1374                    £6,000,000
1397                    13 million
1480                  £2.8 million
1734                  CAD2,000,000
1913     PHP85 million (estimated)
1948                   102,888,900
1953                  3,500,000 DM
1973                    ₤2,300,874
2281                    $14 milion
2451                    ₤6,350,000
3144                   €40 million
3360                $150[6]million
3418                       $218.32
3802                  £4.2 million
3906                           N/A
3959                   760,000 USD
4470                      19 crore
4641                   £17 million
5034               $$200[4]million
5055             $155[2][3]million
5419                 $40[4]million
5424                

In [42]:
budget = budget.str.replace(r'\[\d+\]\s*', '')
budget[~matches_form_one & ~matches_form_two]

136                        Unknown
204     60 millionNorwegian Kroner
478                        Unknown
973                     $34million
1126                   $120million
1226                       Unknown
1278                           HBO
1374                    £6,000,000
1397                    13 million
1480                  £2.8 million
1734                  CAD2,000,000
1913     PHP85 million (estimated)
1948                   102,888,900
1953                  3,500,000 DM
1973                    ₤2,300,874
2281                    $14 milion
2451                    ₤6,350,000
3144                   €40 million
3360                   $150million
3418                       $218.32
3802                  £4.2 million
3906                           N/A
3959                   760,000 USD
4470                      19 crore
4641                   £17 million
5034                  $$200million
5055                   $155million
5419                    $40million
5424                

In [43]:
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [44]:
wiki_movies_df.drop('Budget', axis=1, inplace=True)

In [45]:
release_date=wiki_movies_df['Release date'].dropna().apply(lambda x:''.join(x) if type(x) == list else x)

In [46]:
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
date_form_two = r'\d{4}.[01]\d.[0123]\d'
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
date_form_four = r'\d{4}'

In [47]:
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)

,0
0,"July 11, 1990"
1,"May 17, 1990"
2,"August 10, 1990"
3,"December 25, 1990"
4,"December 19, 1990"
5,"March 22, 1990"
6,"October 6, 1990"
7,"March 8, 1991"
8,"March 7, 1990"
9,"February 23, 1990"


In [48]:
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)


In [49]:
running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [50]:
running_time

0                                             102 minutes
1                                             114 minutes
2                                             113 minutes
3                                             106 minutes
4                                              95 minutes
5                                              95 minutes
6                                             100 minutes
7                                              99 minutes
8                                              50 minutes
9                                                 102 min
10                                             93 minutes
11                                            110 minutes
12                                            126 minutes
13                                            121 minutes
14                                            118 minutes
15                                             99 minutes
16                                             90 minutes
17            

In [51]:
running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False).sum()

6528

In [52]:
running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False)!= True]

9                                                 102 min
26                                                 93 min
28                                                32 min.
34                                                101 min
35                                                 97 min
42                                                91 min.
77                                                91 min.
81                                                 82 min
117     192 minutes (original version) [1] 187 minutes...
126                                              106 min.
128                                               93 min.
143                                               98 min.
144                                              115 min.
146                                               85 mins
171                                              120 min.
197                                               94 min.
218                                               97 min.
223           

In [53]:
running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False).sum()

6877

In [54]:
running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False)!= True]

668                     UK:84 min (DVD version) US:86 min
727                         78-102 min (depending on cut)
840                       Varies (79 [3] –84 [1] minutes)
1347                                              25 : 03
1443    United States: 77 minutes Argentina: 94 minute...
1499                                            1hr 35min
1551                                               varies
1774                    Netherlands:96 min, Canada:95 min
1777                                       approx. 14 min
2273                                           1 h 43 min
2993                                               1h 48m
3925                                              4 hours
4425    US domestic version: 86 minutes Original versi...
4967    Theatrical cut: 97 minutes Unrated cut: 107 mi...
5424                    115 [1] /123 [2] /128 [3] minutes
5447                                    1 hour 32 minutes
7075                Variable; 90 minutes for default path
Name: Running 

In [55]:
running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')


In [56]:
running_time_extract

,0,1,2
0,NaN,NaN,102
1,NaN,NaN,114
2,NaN,NaN,113
3,NaN,NaN,106
4,NaN,NaN,95
5,NaN,NaN,95
6,NaN,NaN,100
7,NaN,NaN,99
8,NaN,NaN,50
9,NaN,NaN,102


In [57]:
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

In [58]:
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row [2]==0 else row[2], axis=1)

In [59]:
wiki_movies_df.drop('Running time', axis=1, inplace=True)

In [60]:
wiki_movies_df

,Based on,Cinematography,Composer(s),Country,Director,Distributor,Editor(s),Language,Producer(s),Production company(s),...,Writer(s),imdb_link,title,url,year,imdb_id,box_office,budget,release_date,running_time
0,"[Characters, by Rex Weiner]",Oliver Wood,"[Cliff Eidelman, Yello]",United States,Renny Harlin,20th Century Fox,Michael Tronick,English,"[Steve Perry, Joel Silver]",Silver Pictures,...,"[David Arnott, James Cappe]",https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,tt0098987,2.140000e+07,20000000.0,1990-07-11,102.0
1,"[the novel, After Dark, My Sweet, by, Jim Thom...",Mark Plummer,Maurice Jarre,United States,James Foley,Avenue Pictures,Howard E. Smith,English,"[Ric Kidney, Robert Redlin]",Avenue Pictures,...,"[James Foley, Robert Redlin]",https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,tt0098994,2.700000e+06,6000000.0,1990-05-17,114.0
2,"[Air America, by, Christopher Robbins]",Roger Deakins,Charles Gross,United States,Roger Spottiswoode,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]","[English, Lao]",Daniel Melnick,"[Carolco Pictures, IndieProd Company]",...,"[John Eskow, Richard Rush]",https://www.imdb.com/title/tt0099005/,Air America,https://en.wikipedia.org/wiki/Air_America_(film),1990,tt0099005,5.771809e+07,35000000.0,1990-08-10,113.0
3,NaN,Carlo Di Palma,NaN,United States,Woody Allen,Orion Pictures,Susan E. Morse,English,Robert Greenhut,NaN,...,Woody Allen,https://www.imdb.com/title/tt0099012/,Alice,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,tt0099012,7.331647e+06,12000000.0,1990-12-25,106.0
4,NaN,Russell Boyd,Maurice Jarre,US,John Cornell,Paramount Pictures,David Stiven,English,John Cornell,NaN,...,Paul Hogan,https://www.imdb.com/title/tt0099018/,Almost an Angel,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,tt0099018,6.939946e+06,25000000.0,1990-12-19,95.0
5,NaN,Jacques Haitkin,Jay Chattaway,United States,Larry Cohen,Triumph Releasing Corporation,"[Claudia Finkle, Armond Leibowitz]",English,"[Larry Cohen, Moctesuma Esparza, Robert Katz]",Epic Productions,...,Larry Cohen,https://www.imdb.com/title/tt0099026/,The Ambulance,https://en.wikipedia.org/wiki/The_Ambulance,1990,tt0099026,NaN,NaN,1990-03-22,95.0
6,NaN,"[Tom Hurwitz, Mathieu Roberts, Nesya Shapiro]",Michael Small,"[United States, United Kingdom]","[Barbara Kopple, Co-directors:, Cathy Caplan, ...",Prestige Films,"[Cathy Caplan, Thomas Haneke, Lawrence Silk]",English,"[Arthur Cohn, Barbara Kopple]","[Cabin Creek, Catholic Communication Campaign,...",...,NaN,https://www.imdb.com/title/tt0099028/,American Dream,https://en.wikipedia.org/wiki/American_Dream_(...,1990,tt0099028,NaN,NaN,1990-10-06,100.0
7,NaN,NaN,NaN,"[United States, Lesotho]",Cedric Sundstrom,Cannon Group,NaN,English,Ovidio G Assonitis,NaN,...,David Geeves,https://www.imdb.com/title/tt0101326/,American Ninja 4: The Annihilation,https://en.wikipedia.org/wiki/American_Ninja_4...,1990,tt0101326,NaN,NaN,1991-03-08,99.0
8,NaN,Bobby Bukowski,Jonathan Sheffer,United States,Deborah Reinisch,Public Broadcasting Service,Jeffrey Wolf,English,"[Sarah Green, Deborah Reinisch]",NaN,...,Terrence McNally,https://www.imdb.com/title/tt0099037/,Andre's Mother,https://en.wikipedia.org/wiki/Andre%27s_Mother,1990,tt0099037,NaN,NaN,1990-03-07,50.0
9,NaN,John LeBlanc,Terry Plumeri,United States,Eric Karson,Imperial Entertainment,Duane Hartzell,English,"[Ash R. Shah, Eric Karson]",NaN,...,S. Warren,https://www.imdb.com/title/tt0099039/,Angel Town,https://en.wikipedia.org/wiki/Angel_Town_(film),1990,tt0099039,8.558100e+05,NaN,1990-02-23,102.0


In [62]:
# File to Load (Remember to change these)
movies_metadata_to_load = "movies_metadata.csv"

# Read the metadata fiel
kaggle_metadata = pd.read_csv(movies_metadata_to_load)


C:\Users\cthompson\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
kaggle_metadata.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [65]:
kaggle_metadata['adult'].value_counts()

False                                                                                                                             45454
True                                                                                                                                  9
 - Written by Ørnås                                                                                                                   1
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
Name: adult, dtype: int64

In [66]:
kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult', axis='columns')

In [68]:
kaggle_metadata['video'].value_counts()

False    45358
True        93
Name: video, dtype: int64

In [69]:
kaggle_metadata['video'] = kaggle_metadata['video'] = 'True'

In [70]:
kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors = 'raise')
kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'],errors='raise')

In [71]:
kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

In [73]:
ratings.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
userId       26024289 non-null int64
movieId      26024289 non-null int64
rating       26024289 non-null float64
timestamp    26024289 non-null int64
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


In [74]:
pd.to_datetime(ratings['timestamp'], unit='s')

0          2015-03-09 22:52:09
1          2015-03-09 23:07:15
2          2015-03-09 22:52:03
3          2015-03-09 22:52:26
4          2015-03-09 22:52:36
5          2015-03-09 23:02:28
6          2015-03-09 22:48:20
7          2015-03-09 22:53:13
8          2015-03-09 22:53:21
9          2015-03-09 23:03:48
10         2015-03-09 22:50:34
11         2015-03-09 22:49:57
12         2015-03-09 23:00:05
13         2015-03-09 22:48:33
14         2015-03-09 23:00:07
15         2015-03-09 22:51:42
16         2015-03-09 22:51:04
17         2015-03-09 23:02:19
18         2015-03-09 23:11:39
19         2015-03-09 23:02:13
20         2015-03-09 23:10:47
21         2015-03-09 23:10:18
22         2015-03-09 22:50:24
23         2015-03-09 22:49:42
24         2015-03-09 23:10:40
25         2015-03-09 22:54:27
26         2015-03-09 22:48:56
27         1997-06-23 04:14:09
28         1997-06-23 04:12:48
29         1997-06-23 04:12:46
                   ...        
26024259   2009-11-01 00:08:48
26024260

In [77]:
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit = 's')

In [78]:
pd.options.display.float_format = '{:20,.2f}'.format
ratings['rating'].plot(kind = 'hist')
ratings['rating'].describe()

count          26,024,289.00
mean                    3.53
std                     1.07
min                     0.50
25%                     3.00
50%                     3.50
75%                     4.00
max                     5.00
Name: rating, dtype: float64